In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
#import config

nbr_boxes=3
nbr_classes=5
cellule_y=12
cellule_x=16
largeur=256
hauteur=192

def block_resnet(input, filters, kernel_size, reduce=False):
    result=layers.Conv2D(filters, kernel_size, strides=1, padding='SAME')(input)
    result=layers.BatchNormalization()(result)
    result=layers.LeakyReLU(alpha=0.1)(result)

    if reduce is True:
        result=layers.Conv2D(filters, kernel_size, strides=2, padding='SAME')(result)
    else:
        result=layers.Conv2D(filters, kernel_size, strides=1, padding='SAME')(result)
        
    if input.shape[-1]==filters:
        if reduce is True:
            shortcut=layers.Conv2D(filters, 1, strides=2, padding='SAME')(input)
        else:
            shortcut=input
    else:
        if reduce is True:
            shortcut=layers.Conv2D(filters, 1, strides=2, padding='SAME')(input)
        else:
            shortcut=layers.Conv2D(filters, 1, strides=1, padding='SAME')(input)
    
    result=layers.add([result, shortcut])
    result=layers.LeakyReLU(alpha=0.1)(result)
    result=layers.BatchNormalization()(result)
    return result

def model(nbr_classes, nbr_boxes, cellule_y, cellule_x):
    entree=layers.Input(shape=(largeur, hauteur, 3), dtype='float32')

    result=block_resnet(entree, 16, 3, False)
    result=block_resnet(result, 16, 3, True)

    result=block_resnet(result, 32, 3, False)
    result=block_resnet(result, 32, 3, True)

    result=block_resnet(result, 64, 3, False)
    result=block_resnet(result, 64, 3, False)
    result=block_resnet(result, 64, 3, True)

    result=block_resnet(result, 128, 3, False)
    result=block_resnet(result, 128, 3, False)
    result=block_resnet(result, 128, 3, True)

    result=layers.Conv2D(nbr_boxes*(5+nbr_classes), 1, padding='SAME')(result)
    sortie=layers.Reshape((cellule_y, cellule_x, nbr_boxes, 5+nbr_classes))(result)

    model=models.Model(inputs=entree, outputs=sortie)

    return model



In [4]:
if __name__ == '__main__':
    # GPU settings
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]